In [112]:
// SETUP
import java.io.File
import java.io.BufferedReader

fun readInputLines(dayIdx: Int): Iterable<String> {
    val fileName = "../data/%02d.txt".format(dayIdx)
    val reader: BufferedReader = File(fileName).bufferedReader()
    return Iterable { reader.lineSequence().iterator() }
}

fun readInputLines(dayIdx: Int, testIdx: Int): Iterable<String> {
    val fileName = "../data/%02dt%d.txt".format(dayIdx, testIdx)
    val reader: BufferedReader = File(fileName).bufferedReader()
    return Iterable { reader.lineSequence().iterator() }
}

In [113]:
fun solution01() {
    val nums = readInputLines(1).map {it.toInt()}

    fun countIncr(nums: Iterable<Int>) = nums.windowed(2).filter {(a, b) -> a < b}.count()

    val res1 = countIncr(nums)
    val res2 = countIncr(nums.windowed(3).map {it.sum()})

    println("Answer 1: $res1\nAnswer 2: $res2")
}

solution01();

Answer 1: 1557
Answer 2: 1608


In [114]:
enum class Command {
  forward, up, down
}

fun parseLine(line: String): Pair<Command, Int> { 
    val (c, n) = line.split(" ")
    return Pair(Command.valueOf(c), n.toInt())
}

fun step1(pos: Pair<Int, Int>, cmdLine: Pair<Command, Int>): Pair<Int, Int> {
    val (x, depth) = pos
    val (cmd, offs) = cmdLine
    return when (cmd) {
        Command.up -> Pair(x, depth - offs)
        Command.down -> Pair(x, depth + offs)
        Command.forward -> Pair(x + offs, depth)
    }
}

fun step2(pos: Triple<Int, Int, Int>, cmdLine: Pair<Command, Int>): Triple<Int, Int, Int> {
    val (x, depth, aim) = pos
    val (cmd, offs) = cmdLine
    return when (cmd) {
        Command.up -> Triple(x, depth, aim - offs)
        Command.down -> Triple(x, depth, aim + offs)
        Command.forward -> Triple(x + offs, depth + aim * offs, aim)
    }
}

fun solution02() {
    val commands = readInputLines(2).map {parseLine(it)}
    val (a1, b1) = commands.fold(Pair(0, 0), ::step1)
    val (a2, b2) = commands.fold(Triple(0, 0, 0), ::step2)
    println("Answer 1: ${a1 * b1}\nAnswer 2: ${a2 * b2}")
}

solution02();

Answer 1: 1728414
Answer 2: 1765720035


In [115]:
fun getMostCommon(nums: Iterable<List<Int>>, pos: Int): Int {
    var n0 = 0;
    var n1 = 0;
    for (num in nums) {
        n0 += 1 - num[pos];
        n1 += num[pos];
    }
    if (n1 >= n0) {
        return 1
    } else {
        return 0
    }
}

fun digitsToDecimal(digits: Iterable<Int>): Int {
    return digits.fold(0) {v, d -> v * 2 + d}
}


fun part1(nums: List<List<Int>>): Int {
    var lc = mutableListOf<Int>()
    var mc = mutableListOf<Int>()
    for (i in 0 until nums[0].size) {
        val m = getMostCommon(nums, i)
        mc.add(m)
        lc.add(1 - m)
    }
    return digitsToDecimal(mc) * digitsToDecimal(lc)
}

fun part2(nums: List<List<Int>>): Int {
    var mnums = (0 until nums.size).toList()
    var lnums = (0 until nums.size).toList()
    for (i in 0 until nums[0].size) {
        if (mnums.size > 1) {
            val m = getMostCommon(mnums.map {nums[it]}, i)
            mnums = mnums.filter {nums[it][i] == m }
        }
        if (lnums.size > 1) {
            val m = getMostCommon(lnums.map {nums[it]}, i)
            lnums = lnums.filter {nums[it][i] == 1 - m }
        }
    }
    return digitsToDecimal(nums[mnums[0]]) * digitsToDecimal(nums[lnums[0]])
}

fun solution03() {
    val nums = readInputLines(3).map {it.toCharArray().map {Character.getNumericValue(it)}}
    println("Answer 1: ${part1(nums)}\nAnswer 2: ${part2(nums)}")
}

solution03()

Answer 1: 2972336
Answer 2: 3368358


In [188]:
typealias Board = MutableList<MutableList<Int>>

fun parseGame(lines: Iterable<String>): Pair<List<Int>, List<Board>> {
    val linesIt = lines.iterator()
    val nums = linesIt.next().split(",").map {it.toInt()}
    linesIt.next()
    
    val boards = mutableListOf<Board>()
    var board = mutableListOf<MutableList<Int>>()
    while (linesIt.hasNext()) {
        val line = linesIt.next()
        if (line.isEmpty()) {
            boards.add(board)
            board = mutableListOf<MutableList<Int>>()
        } else {
            board.add(line.trim().split("\\s+".toRegex()).map {it.toInt()}.toMutableList())
        }
    }
    boards.add(board)
    return Pair(nums, boards)
}

fun applyNum(board: Board, num: Int) {
    for (row in board) {
        row.replaceAll {n -> if (n == num) -1 else n}
    }
}

fun isFull(board: Board, pos: Int, is_vert: Boolean): Boolean {
    for (i in 0 until board.size) {
        val n = if (is_vert) board[i][pos] else board[pos][i]
        if (n != -1) {
            return false;
        }
    }
    return true
}

fun hasWinner(board: Board): Boolean {
    return (0 until board.size)
        .any {isFull(board, it, true) || isFull(board, it, false)}
}

fun runSimulation(game: Pair<List<Int>, List<Board>>): List<Int> {
    val (nums, boardsOrig) = game
    val boards = boardsOrig.map {it.toMutableList()}
    var winners = mutableListOf<Int>()
    var scores = mutableListOf<Int>()
    for (n in nums) {
        for ((i, board) in boards.withIndex()) {
            if (winners.contains(i)) {
                continue
            }
            applyNum(board, n)
            if (hasWinner(board)) {
                val s = board.flatten().filter {it != -1}.sum()
                winners.add(i)
                scores.add(s * n)
            }
        }
    } 
    return scores
}

fun solution04() {
    val scores = runSimulation(parseGame(readInputLines(4)))
    println("Answer 1: ${scores[0]}\nAnswer 2: ${scores.last()}")
}

solution04()

Answer 1: 44736
Answer 2: 1827
